In [75]:
from functools import wraps, lru_cache

#### decorating functions

In [76]:
def eggs(function):
    @wraps(function)
    def _eggs(*args, **kwargs):
        print('%r got args: %r and kwargs: %r' %
              (function.__name__, args, kwargs))
        return function(*args, **kwargs)
    return _eggs


@eggs
def spam(a, b, c):
    '''The spam function Returns a * b + c'''
    return a * b + c

In [77]:
spam(1, 2, 3)

'spam' got args: (1, 2, 3) and kwargs: {}


5

In [78]:
help(spam)

Help on function spam in module __main__:

spam(a, b, c)
    The spam function Returns a * b + c



In [79]:
spam.__name__

'spam'

In [80]:
def spam(eggs):
    return 'spam' * (eggs % 5)


output = spam(3)

In [81]:
def spam(eggs):
    output = 'spam' * (eggs % 5)
    print('spam(%r): %r' % (eggs, output))
    return output


output = spam(3)

spam(3): 'spamspamspam'


#### functools.wraps

In [82]:
def debug(function):
    @wraps(function)
    def _debug(*args, **kwargs):
        output = function(*args, **kwargs)
        print('%s(%r, %r): %r' % (function.__name__, args, kwargs, output))
        return output
    return _debug


@debug
def spam(eggs):
    return 'spam' * (eggs % 5)


output = spam(3)

spam((3,), {}): 'spamspamspam'


#### memoization

In [83]:
def memoize(function):
    function.cache = dict()

    @wraps(function)
    def _memoize(*args):
        if args not in function.cache:
            function.cache[args] = function(*args)
        return function.cache[args]
    return _memoize

@memoize
def fibonacci(n):
    if n < 2:
        return n
    else:
        return fibonacci(n - 1) + fibonacci(n - 2)

for i in range(1, 7):
    print('fibonacci %d: %d' % (i, fibonacci(i)))


fibonacci 1: 1
fibonacci 2: 1
fibonacci 3: 2
fibonacci 4: 3
fibonacci 5: 5
fibonacci 6: 8


In [84]:
fibonacci.__wrapped__.cache

{(1,): 1, (0,): 0, (2,): 1, (3,): 2, (4,): 3, (5,): 5, (6,): 8}

In [85]:
# Create a simple call counting decorator
def counter(function):
    function.calls = 0
    @wraps(function)
    def _counter(*args, **kwargs):
        function.calls += 1
        return function(*args, **kwargs)
    return _counter

# Create a LRU cache with size 3 
@lru_cache(maxsize=3)
@counter
def fibonacci(n):
    if n < 2:
        return n
    else:
        return fibonacci(n - 1) + fibonacci(n - 2)

In [86]:
fibonacci(13)

233

In [87]:
fibonacci.cache_info()

CacheInfo(hits=11, misses=14, maxsize=3, currsize=3)

In [88]:
fibonacci.__wrapped__.__wrapped__.calls


14

#### decorator arguments

In [89]:
def add(extra_n=1):
    'Add extra_n to the input of the decorated function'

    # The inner function, notice that this is the actual
    # decorator
    def _add(function):
        # The actual function that will be called
        @wraps(function)
        def __add(n):
            return function(n + extra_n)

        return __add

    return _add

In [101]:
@add(extra_n=2)
def eggs(n):
    return 'eggs' * n

eggs(1) # for * n = 1 

'eggseggs'

In [91]:
def add(*args, **kwargs):
    'Add n to the input of the decorated function'

    # The default kwargs, we don't store this in kwargs
    # because we want to make sure that args and kwargs
    # can't both be filled
    default_kwargs = dict(n=1)

    # The inner function, notice that this is actually a
    # decorator itself
    def _add(function):
        # The actual function that will be called
        @wraps(function)
        def __add(n):
            default_kwargs.update(kwargs)
            return function(n + default_kwargs['n'])

        return __add

    if len(args) == 1 and callable(args[0]) and not kwargs:
        # Decorator call without arguments, just call it
        # ourselves
        return _add(args[0])
    elif not args and kwargs:
        # Decorator call with arguments, this time it will
        # automatically be executed with function as the
        # first argument
        default_kwargs.update(kwargs)
        return _add
    else:
        raise RuntimeError('This decorator only supports '
                           'keyword arguments')

@add
def spam(n):
    return 'spam' * n

@add(n=3)
def eggs(n):
    return 'eggs' * n

In [92]:
spam(3)

'spamspamspamspam'

In [93]:
eggs(2)

'eggseggseggseggseggs'

In [95]:
@add(n=4)
def bacon(n):
    return 'bacon' * n

In [96]:
bacon(2)

'baconbaconbaconbaconbaconbacon'